## affine gap penalty string distance

> input: food names

> output: words to remove / synomyms

In [1]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [2]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180309'

In [3]:
## output from food synset entities
file_name = "food_entities__20180308.p"
df_retrived = f.retrive_file(file_name)

retrived: food_entities__20180308.p


In [4]:
print df_retrived.shape
df_retrived.head(2)

(92298, 12)


,index,food_name,locs,synonyms,menu_names,restaurant,review,review.clean_text,review.restaurant,review.text,review.image,short_name
0,food_0,-PRON- ann,"[deliveroo/food/68155, deliveroo/food/AV062oM7...",[em ann],"[(Em Ann M., 3)]","[delivery_3748, delivery_3749, delivery_3750]",[burppleinitial/review/vGui2G8q],[maggie mee goreng from the rooftop at screeni...,"[(burpple_12581, 1)]",[Maggi Mee Goreng from The Rooftop at Screenin...,[vGui2G8q],-PRON- ann
1,food_1,-PRON- bai vege,"[foodpanda/menu_item/AV44Ejd8W4PLiMnQOaiu, foo...",[you bai vege],"[(H6. You Bai Vege, 2)]","[delivery_221, delivery_222]",NaN,NaN,NaN,NaN,NaN,-PRON- bai vege


In [5]:
# food names without drinks
food_names = df_retrived["food_name"].tolist()

import pandas as pd 
try:
    file_name = "drink_20180308.csv"
    drink_lst = pd.read_csv(file_name, header=None)[0].tolist()
except:
    drinl_lst = []
drink_lst = [""] + drink_lst

food_names = [s for s in sorted(list(set(food_names))) if s not in drink_lst]
print len(food_names)

92077


In [6]:
# count frequency of occurance of the unigram tokens
tokens = [s.split() for s in food_names]
from collections import Counter
c = Counter([item for sublist in tokens for item in sublist])
count_tokens = sorted(c.items(), key=lambda x: -x[1])
count_tokens[:5]

[(u'with', 16055),
 (u'chicken', 11071),
 (u'fry', 8164),
 (u'rice', 6981),
 (u'and', 6641)]

In [7]:
# all tokens
unique_tokens = [s[0] for s in count_tokens]
print "number of uinique tokens: %d"%len(unique_tokens)

number of uinique tokens: 14603


# Use affine gap penalty string distance

In [8]:
# example 
import affinegap
d1 = affinegap.affineGapDistance('foo', 'bar')
d2 = affinegap.affineGapDistance('foo', 'bar',
                                 matchWeight = 1,
                                 mismatchWeight = 11,
                                 gapWeight = 10,
                                 spaceWeight = 7,
                                 abbreviation_scale = .125)
d3 = affinegap.normalizedAffineGapDistance('foo', 'bar')

print d1, d2, d3

33.0 33.0 5.5


In [9]:
sample_names = food_names[140:150]
sample_names 

[u'absolutely mango yogurt cake',
 u'abundance',
 u'abura soba',
 u'aburasoba',
 u'aburi akami maguro',
 u'aburi bara chirashi',
 u'aburi bara chirashi don set',
 u'aburi broccoli',
 u'aburi cha shu',
 u'aburi chicken raman']

In [10]:
sample_dist = []
sample_dist_norm = []
import itertools
for s0, s1 in itertools.combinations(sample_names,2):
    sample_dist.append([affinegap.affineGapDistance(s0, s1), s0, s1])
    sample_dist_norm.append([affinegap.normalizedAffineGapDistance(s0, s1), s0, s1]) 
sample_dist = sorted(sample_dist, key=lambda x: x[0])
sample_dist_norm = sorted(sample_dist_norm, key=lambda x: x[0])

In [11]:
sample_dist[:5]

[[26.0, u'abura soba', u'aburasoba'],
 [27.25, u'aburi bara chirashi', u'aburi bara chirashi don set'],
 [53.75, u'aburasoba', u'aburi cha shu'],
 [58.125, u'aburasoba', u'aburi akami maguro'],
 [59.125, u'abura soba', u'aburi bara chirashi']]

In [12]:
sample_dist_norm[:5]

[[0.592391312122345, u'aburi bara chirashi', u'aburi bara chirashi don set'],
 [1.3684210777282715, u'abura soba', u'aburasoba'],
 [1.787162184715271, u'abura soba', u'aburi bara chirashi don set'],
 [2.0387930870056152, u'abura soba', u'aburi bara chirashi'],
 [2.0487804412841797, u'aburi bara chirashi don set', u'aburi broccoli']]

In [15]:
%%time
# to estimate the total time
dist_norm = []
import itertools
for s0, s1 in itertools.combinations(food_names[1000:2000],2):
    try:
        d = affinegap.normalizedAffineGapDistance(s0, s1)
    except: 
        d = -1
    dist_norm.append([d, s0, s1])
dist_norm = sorted(dist_norm, key=lambda x: x[0])
print "dist pairs: %d"%len(dist_norm)
threshold = 1
res = [l for l in dist_norm if (l[0] <threshold and l[0]>0.5)]
print "dist smaller than threshold=%d: %d"%(threshold, len(res))

dist pairs: 499500
dist smaller than threshold=1: 509
CPU times: user 3.77 s, sys: 176 ms, total: 3.94 s
Wall time: 3.75 s


In [16]:
# save the file back to words to remove
import myfunctions as f
file_name = "dist_norm_" + f.today() + ".csv"
import csv
with open(file_name, "wb") as output:
    writer = csv.writer(output)
    writer.writerows(res) 
print ("saved: %s" % file_name)

saved: dist_norm_20180309.csv


In [17]:
%%time
# estimated 8 hours = 92077*92077/1000/1000*3.77/3600
dist_norm = []
import itertools
for s0, s1 in itertools.combinations(food_names,2):
    try:
        d = affinegap.normalizedAffineGapDistance(s0, s1)
        if d>0.5 and d<1:
            dist_norm.append([d, s0, s1])
    except: 
        pass
dist_norm = sorted(dist_norm, key=lambda x: x[0])
print "dist smaller than threshold=%d: %d"%(threshold, len(dist_norm))

dist smaller than threshold=1: 102848
CPU times: user 6h 22min 52s, sys: 2min 54s, total: 6h 25min 46s
Wall time: 6h 21min 36s


In [18]:
# save the file back to words to remove
import myfunctions as f
file_name = "dist_norm_" + f.today() + ".csv"
import csv
with open(file_name, "wb") as output:
    writer = csv.writer(output)
    writer.writerows(dist_norm) 
print ("saved: %s" % file_name)

saved: dist_norm_20180309.csv


>  Example pair 

> 0.559782624,	aloo mutter sukhewali,	aloo mutter sukhewali ali

    consider multi-filed matching
    -> description similarity > some desc_sim threshold
    -> offered by the same restaurants 


In [49]:
%reload_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180313'

In [50]:
size = 50
sample_lst = [l[1:] for l in dist_norm[:size]]
additional_words, lst = f.parallel_detection(sample_lst )

In [51]:
additional_words

[(u'set', 20),
 (u'fry', 3),
 (u'pan', 3),
 (u'france', 2),
 (u'cl', 1),
 (u'bento', 1),
 (u'nut', 1),
 (u'pc', 1),
 (u'veneto', 1),
 (u'soup', 1),
 (u'basket', 1),
 (u'new', 1),
 (u'rice', 1)]

In [52]:
lst

{frozenset({u'sauc', u'sauce'}),
 frozenset({u'sandwic', u'sandwich'}),
 frozenset({u'aglio', u'set'}),
 frozenset({u'keema', u'keemah'}),
 frozenset({u'chill', u'chilli'}),
 frozenset({u'toast', u'toaste'}),
 frozenset({u'bit', u'bite'}),
 frozenset({u'oat', u'oats'}),
 frozenset({u'web', u'webs'}),
 frozenset({u'pc', u'pcs'}),
 frozenset({u'consomm', u'consomme'}),
 frozenset({u'pan fry', u'pcs steam and pcs sweet garlic'})}

In [ ]:
""" complete set with affine distance treshold = 0.6"""

In [56]:
size = len(dist_norm)
threshold = 0.6
sample_lst = [l[1:] for l in dist_norm[:size] if l[0] <threshold]
print len(sample_lst)
words_rmv, syn_lst = f.parallel_detection(sample_lst )

6996


In [57]:
words_rmv[:20]

[(u'rice', 822),
 (u'set', 776),
 (u'soup', 281),
 (u'with', 242),
 (u'egg', 185),
 (u'and', 184),
 (u'pasta', 157),
 (u'meal', 152),
 (u'fry', 147),
 (u'sauce', 146),
 (u'salad', 137),
 (u'noodle', 122),
 (u'pizza', 110),
 (u'bento', 97),
 (u'don', 86),
 (u'wing', 81),
 (u'dry', 79),
 (u'in', 73),
 (u'chop', 63),
 (u'wrap', 63)]

In [60]:
len(words_rmv)

821

In [58]:
syn_lst

{frozenset({u'mentai', u'mentaiko'}),
 frozenset({u'frit', u'fritto'}),
 frozenset({u'cuttlefish', u'sambal'}),
 frozenset({u'hamburg', u'hamburger'}),
 frozenset({u'and chicken', u'bento'}),
 frozenset({u'chilli crab sauce', u'fry'}),
 frozenset({u'lemak', u'lemakk'}),
 frozenset({u'kulcha', u'kulchan'}),
 frozenset({u'salmon', u'salmone'}),
 frozenset({u'oat', u'oats'}),
 frozenset({u'aglio', u'spaghetti'}),
 frozenset({u'jalfrezi', u'jalfrezie'}),
 frozenset({u'egg', u'rice with sirloin steak'}),
 frozenset({u'aglio', u'pasta set'}),
 frozenset({u'bai', u'shrimp'}),
 frozenset({u'floss', u'flossant'}),
 frozenset({u'grill', u'grille'}),
 frozenset({u'burger', u'doo'}),
 frozenset({u'sauc', u'sauce'}),
 frozenset({u'chettina', u'chettinad'}),
 frozenset({u'egg', u'rice with chicken wing'}),
 frozenset({u'moo or gai', u'prawn'}),
 frozenset({u'aglio', u'thai sauce'}),
 frozenset({u'steam', u'steamfish'}),
 frozenset({u'gravy', u'seafood and egg'}),
 frozenset({u'pc', u'pcs'}),
 frozen

In [59]:
len(syn_lst)

228